# Guts notebook

![guts](assets/img/guts.png)

This datasets contains stomach contents from sardines and sprats in the Bay of Biscay.

- In total there are 144 samples (too many).

- There are 5 groups of samples:
  - break: sardines from the shelf break (A)
  - outer: sardines from hte outer shelf (B)
  - day: sardines from the inner shelf taken by the day
  - night: sardines from the inner shelf taken at night
  - sprat: sprats from the inner shelf

- But we don't want to study bacteria but the zooplankton they eat. Therefore we don't use the 16S marker but the 18S.



Your task is to study the differences in the diet between a pair of sample types.

This is who has to do what:

| Student   | Pair of samples | sample_data      |
|-----------|-----------------|------------------|
| Hossain   | Break vs night  | guts_break_night |
| Gema      | Break vs outer  | guts_break_outer |
| Madeleine | Break vs sprat  | guts_break_sprat |
| Samuel    | Night vs outer  | guts_night_outer |
| Borja     | Night vs sprat  | guts_night_sprat |
| Gaetán    | Outer vs sprat  | guts_outer_sprat |

Feel free to have last week's notebooks open in another tab.

The files you have to analyze are in the nervion folder:

In [1]:
ll -h guts

total 48M
drwxr-xr-x 2 jovyan jovyan 4.0K Mar 31 14:31 ./
drwxr-xr-x 1 jovyan jovyan 4.0K Apr 19 18:28 ../
-rw-r--r-- 1 jovyan jovyan 3.6M Mar 31 14:31 barcodes.fastq.gz
-rw-r--r-- 1 jovyan jovyan  23M Mar 31 14:31 forward.fastq.gz
-rw-r--r-- 1 jovyan jovyan  22M Mar 31 14:31 reverse.fastq.gz


It is a decaffeinated dataset from the one from the article (~ 0.5 to 1%) so you can analyze in a moment, rather than waiting a day or two to then realize that you did it wrong.

Insteado of sequencing in a single go the 16S sequence, we sequence twice the fragment, once in the 5' to 3' direction (`forward.fastq.gz`), and again in the 3' to 5' sense (`reverse.fastq.gz`). By sequencing twice we get more accurate measurements. 

## Warning

The only difference with the dataset from the previous session is that we are using Illumina **Paired**-End sequencing instead of **Single**-End.

Therefore, you will need to modify the commands specific to single-end to paired end, i.e.:
- `qiime demux emp-single` now becomes `qiime demux emp-paired`
- `qiime dada2 denoise-single` is now `qiime dada2 denoise-paired`

Appart from that, almost every command is the same.

The purpose of this exercise is to see that you can follow from the start to the end  

## Tips

- Save often, and download the notebook to your laptop from time to time.

- Remember to `pause` your notebook if you think you need to rest or to think.

- Click on the Fast Forward button from time to time (between restart and download buttons in the top bar) to see that everything works from beginning to end

You are expected to type commands in code cells like the following one with a `[ ]` on the side

and text in the cells without the `[ ]`:

If you find two or more code cells together, is because you are being asked for more than one command. Put each command in different cells

# Exercises

## 1. Samples

Let's see if you understand the contents of your `sample-data.tsv` file.

1.1 How many samples are in your `sample-data.tsv` file?

There are 43 samples (guts_outer)

1.2 How many are for each city?

15 samples from Gironde, and 27 from Capbreton

1.3 How many samples are for each season of the year?

15 samples of day and 27 of night

1.4 What name has the column that holds the barcode of the sample?

BarcodeSequence

## 2. Pre-processing

2.1 Import your dataset to Qiime2. It is not of the type `EMPSingleEndSequences`.

In [2]:
qiime tools import \
  --type        EMPPairedEndSequences \
  --input-path  guts \
  --output-path sequences.qza

Imported guts as EMPPairedEndDirFmt to sequences.qza



2.2 Demultiplex your sequences with `demux`. Remember that we are not doing Paired-End sequencing. Also, you need to pass the `--p-rev-comp-mapping-barcodes` parameter.

In [3]:
qiime demux emp-paired \
    --i-seqs                     sequences.qza \
    --m-barcodes-file            guts_night_outer.tsv \
    --m-barcodes-column          BarcodeSequence \
    --p-rev-comp-mapping-barcodes \
    --o-per-sample-sequences     demux.qza \
    --o-error-correction-details demux-details.qza
   

Saved SampleData[PairedEndSequencesWithQuality] to: demux.qza
Saved ErrorCorrectionDetails to: demux-details.qza



2.3 Generate the `demux.qzv`file and visualize it in [view.qiime.org](view.qiime.org)

In [4]:
qiime demux summarize \
    --i-data          demux.qza \
    --o-visualization demux.qzv

Saved Visualization to: demux.qzv



2.4 How many samples contains more than 1,000 reads?

There are 38 samples that have more than 1000 reads

2.5 What is the name of the smallest sample?

sardine_gironde_inner_night_L14.1 (178)

2.6 And the biggest?

sardine_gironde_inner_night_K95.10.1 (4937)

Click the "Interactive Quality Plot" tab.

2.7 What is the median length of the forward reads?

151 nts

2.8 And the mean quality of the forward reads?

38

2.9 What is the median length of the reverse reads?

151 nts

2.10 And the mean quality of the reverse reads?

39

2.11 Do these reads need to be trimmed either at the 5' or 3' end in order to get a high quality experiment? High quality means a score above 30 (probability of error < 0.001),

There is no need since both reads have a quality larger than 30

2.12 Perform error correction with DADA2

In [5]:
qiime dada2 denoise-paired \
    --i-demultiplexed-seqs       demux.qza \
    --p-trunc-len-f                151 \
    --p-trunc-len-r                151 \
    --o-representative-sequences rep-seqs.qza \
    --o-table                    table.qza   \
    --o-denoising-stats          dada2-stats.qza
     

Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza
Saved SampleData[DADA2Stats] to: dada2-stats.qza



2.13 Convert the feature table (`table.qza`) to qzv and open it in the visualizer.

In [6]:
qiime feature-table summarize \
    --i-table                table.qza \
    --m-sample-metadata-file guts_night_outer.tsv \
    --o-visualization        table.qzv

Saved Visualization to: table.qzv



2.14 How many ASVs have you found?

111

2.15 How many reads are in total after error correction?

63139

2.16 How many reads has the biggest sample?

3202, sardine_capbreton_outer_day_L36.10.1

2.17 How many reads has the smallest sample?

5, sardine_capbreton_outer_day_L62

2.18 How many samples have more than 1,000 reads?

30

2.19 How many ASVs have more than 1,000 reads?

7

2.20 In how many samples is present the most frequent ASV? 

39

2.21 What name has this ASV?

445008ecee377be1d073769dbc4e6d0a

2.22 Convert the `representative-sequences.qza` file to `qzv`.

In [7]:
qiime feature-table tabulate-seqs \
  --i-data          rep-seqs.qza \
  --o-visualization rep-seqs.qzv

Saved Visualization to: rep-seqs.qzv



2.23 How long is the longest ASV?

The longest ASV is 210 nucleotides long

2.24 And the shortest?

The shortest ASV is 153 nucleotides long

2.25 How long is the most common ASV?

174 nucleotides long

2.26 Click on the sequence and perform a BLAST search. What is the first result?

**Note**: this step will take a while. Come back later to finish this.

Junceella aquamata 18S ribosomal RNA gene, partial sequence, AY962535.1

2.27 Visualize the `dada2-stats` file.

In [8]:
qiime metadata tabulate \
    --m-input-file    dada2-stats.qza \
    --o-visualization dada2-stats.qzv

Saved Visualization to: dada2-stats.qzv



2.28 Which sample has been filtered the most in terms of percentages? Type the sample name and the percentage.

sardine_capbreton_outer_day_XX11 (77.44)

2.29 And the least?

sardine_gironde_inner_night_L14 (3.82)

It should be a great idea to save your notebook to your laptop, press the Fast Forward button to see that all files generate properly, `pause` the notebook, and take 5 minutes away from the screen.

In [ ]:
pause

## 3. Taxonomy

In the folder databases you have two trained sequence classifiers:
- greengenes, which contains 16S sequences 
- metazoogene, which contain 18S sequences

In [ ]:
ll databases/

3.1 Which one is appropriate to analyze the eukaryotes present in the guts of these fish?

18S 

3.2 Classify your sequences with that database.

In [ ]:
qiime feature-classifier classify-sklearn \
    --i-classifier     databases/metazoogene.qza \
    --i-reads          rep-seqs.qza \
    --o-classification taxonomy.qza

3.3 Convert the results into a visualization

In [ ]:
qiime metadata tabulate \
    --m-input-file    taxonomy.qza \
    --o-visualization taxonomy.qzv

3.4 How many ASVs have been classified as Animalia and no phylum?

35

3.5 How many belong to the class Copepoda?

16

3.6 Draw the taxonomic barplot.

In [ ]:
qiime taxa barplot \
  --i-table         table.qza \
  --i-taxonomy      taxonomy.qza \
  --m-metadata-file guts_night_outer.tsv \
  --o-visualization taxa-bar-plots.qzv

3.7 What sample contains the most of the Engraulis encrasicolus species?

sardine_capbreton_outer_day_L31.10 (3.130%)

3.8 What is the most common phylum (in average)?

Arthropoda

## 4. Diversity

4.1 Create first the phylogeny, and then compute the core metrics. Filter out samples with less than 1,000 reads.

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences        rep-seqs.qza \
    --o-alignment        phylo-aligned-seqs.qza \
    --o-masked-alignment phylo-masked-aligned-seqs.qza \
    --o-tree             phylo-unrooted-tree.qza \
    --o-rooted-tree      phylo-rooted-tree.qza

In [ ]:
export UNIFRAC_USE_GPU=N  # There is an error if you have a GPU
qiime diversity core-metrics-phylogenetic \
    --i-phylogeny      phylo-rooted-tree.qza \
    --i-table          table.qza \
    --p-sampling-depth 1000 \
    --m-metadata-file  guts_night_outer.tsv \
    --output-dir       metrics

4.2 Compute the alpha-rarefaction, using a maximum depth of 2000 sequences.

In [ ]:

qiime diversity alpha-rarefaction \
    --i-table         table.qza \
    --i-phylogeny     phylo-rooted-tree.qza \
    --p-max-depth     2000 \
    --m-metadata-file guts_night_outer.tsv \
    --o-visualization metrics/alpha-rarefaction.qzv

4.3 According to the Shannon index, do we have almost the same community information at a sequencing depth of 1,000 and 2,000?

There is a similar Shannon diversity for most of the barcodes, although some are not represented at higher sequence depths.

4.4 Which barcode has the least Shannon diversity?

CTCCCTTTGTGT

4.5 Compute the evenness of the samples.

In [ ]:
# evenness
qiime diversity alpha-group-significance \
  --i-alpha-diversity metrics/evenness_vector.qza \
  --m-metadata-file   guts_night_outer.tsv \
  --o-visualization   metrics/evenness-group-significance.qzv

4.6 Do the same for the richness.

In [ ]:
# Richness ~ faith phylogenetic diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity metrics/faith_pd_vector.qza \
  --m-metadata-file   guts_night_outer.tsv \
  --o-visualization   metrics/faith-pd-group-significance.qzv

4.7 Is the richness different between your types of fish? What is your p-value?

The richness is different since H: 7.806, p:0.005 > (0.05)

4.8 Is the evenness different between fish type? What is your p-value?

The evenness is different since H: 6.668, p: 0.001 > (0.05)

4.9 Compute the beta diversity using the Weighted Unifrac distance matrix, grouping by fish type.

In [ ]:

qiime diversity beta-group-significance \
    --i-distance-matrix metrics/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file   guts_night_outer.tsv \
    --m-metadata-column sample_type \
    --o-visualization   metrics/weighted-unifrac-body-site-significance.qzv \
    --p-pairwise
     

4.10 According to the pairwise Permanova test, are there differences between fish types?

Yes there are differences, test statistic: 9.05, p-value: 0.001> (0.05)

`pause` and take 5 minutes if you need it. Maybe it is time to collect that BLAST result from before.

In [ ]:
pause

## 5 Analysis of Composition with ANCOM

5.1 Do the analysis of composition at the ASV level, and compare the differences of composition between **seasons**. You have to perform the `add-pseudocount` step first.

In [ ]:
qiime feature-table filter-samples \
    --i-table          table.qza \
    --m-metadata-file  guts_night_outer.tsv \
    --o-filtered-table region.qza

In [ ]:

qiime composition add-pseudocount \
  --i-table             region.qza \
  --o-composition-table comp-region.qza

In [ ]:
qiime composition ancom \
    --i-table           comp-region.qza \
    --m-metadata-file   guts_night_outer.tsv \
    --m-metadata-column region \
    --o-visualization   ancom-region.qzv

5.2 How many ASVs are statistically different? 

11

5.3 How many have a positive CLR (centered log ratio)?

6

5.4 How many are negative?

5

5.5 Colapse the ASV table so we can compute the results at the family level

In [ ]:
qiime taxa collapse \
  --i-table           region.qza \
  --i-taxonomy        taxonomy.qza \
  --p-level           5 \
  --o-collapsed-table gut-table-5.qza

5.6  Compute the pseudocounts for that table, and run the ANCOM analysis again between seasons.

In [ ]:
qiime composition add-pseudocount \
  --i-table             gut-table-5.qza \
  --o-composition-table comp-gut-table-5.qza

In [ ]:
qiime composition ancom \
  --i-table           comp-gut-table-5.qza \
  --m-metadata-file   guts_night_outer.tsv \
  --m-metadata-column region \
  --o-visualization   region-ancom-subject.qzv

5.7  How many elements are statistically different now?

7

## End of the notebook

Make sure that
- all the text cells contain only text
- all the code cells contain only code
- That the entire notebook runs from the beginning to the end without errors by pressing the Fast Forward button.